### module configuration

In [20]:
import torch
import torchvision 
import torchvision.transforms as transforms
import pdb
import torch.nn as nn

### device -> GPU

In [21]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### hyperparameter 지정

In [22]:
num_epoch = 5

learning_rate = 0.001

batch_size = 100
#batch 해야하는 이유?
#로스 조정을 여러번하기 위함.
#컴퓨터 부하를 줄이기 위함(병렬처리 하니까)

num_classes = 10 
#답이 열개 



### Dataloader

In [23]:
#step1

train_dataset = torchvision.datasets.FashionMNIST(download=True,
                                                  root="../../data",
                                                  train=True, #트레이닝 데이터라는 의미
                                                  transform = transforms.ToTensor() #이미지 변환, 스케일링, 채널 변경
                                                  )

test_dataset = torchvision.datasets.FashionMNIST(
                                                  root="../../data",
                                                  train=False, #테스트 데이터라는 의미
                                                  transform = transforms.ToTensor() #이미지 변환, 스케일링, 채널 변경
                                                  )

#step2 

train_dataloader = torch.utils.data.DataLoader(batch_size=batch_size,
                                               shuffle = True,
                                               dataset = train_dataset
                                               )

test_dataloader = torch.utils.data.DataLoader(batch_size=batch_size,
                                               shuffle = False,
                                               dataset = test_dataset
                                               )

### Model Template


In [24]:
class ConvNet(nn.Module):
    def __init__(self,num_classes): #정답의 갯수가 몇개인가를 선지정해준다. 
        super(ConvNet,self).__init__()
        
        #레이어 설정
        self.layer1 = nn.Sequential(
            #convolution 
            nn.Conv2d(1,16,kernel_size=3,padding=1,stride=1) , 
            #activation function
            nn.ReLU()
        )

        self.layer2 = nn.MaxPool2d(kernel_size=2,stride=2) # 14 

        self.layer3 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=3,padding=1,stride=1) ,
            nn.ReLU()
        )
        self.layer4 = nn.MaxPool2d(kernel_size=2,stride=2) # 7

        self.layer5 = nn.Linear(7*7*32,num_classes)
    
    def forward(self,x):
        #출력물
        out = self.layer1(x) #x는 이미지 데이터.  #getter setter?
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        #reshape! 
        #pdb.set_trace()
        out = out.reshape(out.size(0),-1) #.. 
        out = self.layer5(out)
        return out


### 모델 선정의

In [25]:
model = ConvNet(num_classes).to(device) #GPU 
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### 학습(training)

In [30]:
total_step = len(train_dataloader) #배치사이즈의 수 만큼 돌리는 것을 하나의 epoch으로 볼 것이다.

for epoch in range(num_epoch):
    for i, (images,labels) in enumerate(train_dataloader):
        image = images.to(device) #이미지 연산을 GPU로... 병렬처리 해주겠다?
        label = labels.to(device) # image가 gpu서버에서 연산되므로... 같이가자... 

        output_forward = model(image)

        loss_w = loss_function(output_forward,label) 
        #정답과 비교 + 로스값 계산 + 양수화 시키고 증폭... log로 숫자 다시 낮추고... 확률 계산... 돌려준다. 

        optimizer.zero_grad() #초기화. 

        loss_w.backward()

        optimizer.step()

        if (i+1) % 100 == 0: 
            print("Epoch : [{}/{}], Step : [{}/{}], Loss:{:.3f}".format(epoch+1,num_epoch,i+1,total_step,loss_w.item()))



Epoch : [1/5], Step : [100/600], Loss:0.230
Epoch : [1/5], Step : [200/600], Loss:0.187
Epoch : [1/5], Step : [300/600], Loss:0.191
Epoch : [1/5], Step : [400/600], Loss:0.177
Epoch : [1/5], Step : [500/600], Loss:0.244
Epoch : [1/5], Step : [600/600], Loss:0.087
Epoch : [2/5], Step : [100/600], Loss:0.087
Epoch : [2/5], Step : [200/600], Loss:0.163
Epoch : [2/5], Step : [300/600], Loss:0.160
Epoch : [2/5], Step : [400/600], Loss:0.205
Epoch : [2/5], Step : [500/600], Loss:0.232
Epoch : [2/5], Step : [600/600], Loss:0.205
Epoch : [3/5], Step : [100/600], Loss:0.186
Epoch : [3/5], Step : [200/600], Loss:0.131
Epoch : [3/5], Step : [300/600], Loss:0.237
Epoch : [3/5], Step : [400/600], Loss:0.146
Epoch : [3/5], Step : [500/600], Loss:0.152
Epoch : [3/5], Step : [600/600], Loss:0.130
Epoch : [4/5], Step : [100/600], Loss:0.095
Epoch : [4/5], Step : [200/600], Loss:0.096
Epoch : [4/5], Step : [300/600], Loss:0.153
Epoch : [4/5], Step : [400/600], Loss:0.285
Epoch : [4/5], Step : [500/600],

### 평가

In [41]:
correct = 0
total = 0

with torch.no_grad():
    #평가를 하는 것이므로 경사구하는 것을 하지 않는다.
    for images,labels in test_dataloader:
        
        image = images.to(device)
        labels = labels.to(device)

        outputs = model(image) #예측을 해본다. 

        _, predicted = torch.max(outputs.data,1) 

        #1의 의미. 각 '행'에서 최고인 값의 인덱스값을 주겠다는 의미가 된다.
        #_은 value, predicted은 해당 value의 index값이 된다. 
        """ 
        a = torch.randn(4, 4)
        >>>a
        tensor([[-1.2360, -0.2942, -0.1222,  0.8475],
                [ 1.1949, -1.1127, -2.2379, -0.6702],
                [ 1.5717, -0.9207,  0.1297, -1.8768],
                [-0.6172,  1.0036, -0.6060, -0.2432]])
        >>> torch.max(a, 1)
        torch.return_types.max(values=tensor([0.8475, 1.1949, 1.5717, 1.0036]), indices=tensor([3, 0, 0, 1]))
        
        """

        total += labels.size(0)
        
        correct += (predicted == labels).sum().item() 
    print('Test Accuracy of the model on the 10000 test images:{}%'.format(100*correct /total))
    torch.save(model.state_dict(), 'model.ckpt') 

    

Test Accuracy of the model on the 10000 test images:90.78%
